In [7]:
%load_ext autoreload
%autoreload 2

import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from datetime import datetime, timezone, tzinfo

from pyproj import Proj, CRS, Transformer

from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track, read_segypos

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

In [65]:
def read_opencpn_csv(path):
    
    with open(path, 'r') as f1:
        f1_content = f1.read().splitlines()
        
        line_num = len(f1_content)
        
        f1_array = np.zeros((4, len(f1_content[1:])))
        
        for num, line in enumerate(f1_content[1:]):
            line_content = line.split(',')
            
            f1_array[0,num] = num
            f1_array[1,num] = datetime.timestamp(datetime.strptime(line_content[4],'%Y/%m/%d %H:%M:%S+00'))
            f1_array[2,num] = float(line_content[5])
            f1_array[3,num] = float(line_content[6])
            
            
            # calculate bearing
            
        
    return f1_array
            

In [72]:
path = r'C:\YandexDisk\aa_cloudmbesproceessing\AI-68\track_processing\OpenCPN_track_StationPC_AlmostFinal_(SvetlChannel).csv'

track_array = read_opencpn_csv(path)

In [73]:
diff_array = np.diff(track_array[1])

save_to = r'C:\YandexDisk\aa_cloudmbesproceessing\AI-68\track_processing\OpenCPN_track_StationPC_AlmostFinal_(SvetlChannel)_proc.csv'

with open(save_to, 'w') as f2:
    f2.write('num,timestamp,x,y,diff,group,datetime\n')
    
    group = 0
    
    for num, time in enumerate(track_array[1,:]):
        f2.write(f'{track_array[0,num]},{track_array[1,num]},')
        f2.write(f'{track_array[2,num]},{track_array[3,num]},')
        
        if num > 0:
            f2.write(f'{diff_array[num-1]},')
            
            if diff_array[num-1] > 2000:
                group += 1
            elif diff_array[num-1] < 0:
                group += 1
        else:
            f2.write('0,')
        f2.write(f'{group},{datetime.strftime(datetime.fromtimestamp(track_array[1,num], timezone.utc), '%Y-%m-%dT%H:%M:%S')}\n')
                
